In [14]:
%load_ext autoreload
%autoreload 1
%aimport load
%aimport PreProcess
fmri_lag = 4
freq_hz = 15
frames = 108000
extracted_features_path = "./extracted_feature.npy"
extracted_features_path_test = "./extracted_feature_test.npy"

import numpy as np
import tensorflow as tf

from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

from keras.models import Model
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras import regularizers



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
num_responses = frames - fmri_lag * freq_hz
print(f'number of frames/response values: {num_responses}')

number of frames/response values: 107940


In [4]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = 'x'):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()
        

In [5]:
### Response ###
resp = load.load_train_response_all("1")

In [6]:
chop_resp = resp[:, fmri_lag : ]
print(f'ch: {chop_resp.shape}')

ch: (73728, 7196)


In [7]:
chop_resp = np.transpose(chop_resp)
print(f'chopped response tranposed with shape {chop_resp.shape}')

chopped response tranposed with shape (7196, 73728)


In [8]:
upsampled = PreProcess.upsample(chop_resp, num_responses)
print(f'upsampled shape: {upsampled.shape}')

upsampled shape: (107940, 73728)


In [9]:
v1lh = load.load_train_response("1", "v1lh")

In [32]:
!cd loaded
np.save("v1lh_train_response.npy", np.asarray(v1lh))
print(f'v1lh loaded with shape {v1lh.shape}')

v1lh loaded with shape (490, 7200)


In [11]:
v1lh = PreProcess.dropna(v1lh)
v1lh.shape

(490, 7200)

In [12]:
chop_resp = v1lh[:, fmri_lag : ]
print(f'ch: {chop_resp.shape}')

ch: (490, 7196)


In [15]:
chop_resp = np.transpose(chop_resp)
print(f'chopped response tranposed with shape {chop_resp.shape}')

chopped response tranposed with shape (7196, 490)


In [16]:
upsampled = PreProcess.upsample(chop_resp, num_responses)
print(f'upsampled shape: {upsampled.shape}')

upsampled shape: (107940, 490)


In [17]:
def tikreg_model(X, Y):
	# Specify fit
	options = dict(ridges=np.logspace(0,3,11), weights=True, metric='rsquared')
	fit = models.cvridge(X[0], Y[0], X[1], Y[1], **options)

	# Evaluate results
	weights_estimate = fit['weights']
	return (fit['cvresults'].shape) # (5, 1, 11, 2): (nfolds, 1, nridges, nresponses)

In [18]:
clf = MultiOutputRegressor(ElasticNet(random_state=0))

In [ ]:
train_features = np.load(extracted_features_path)

In [23]:
train_features = train_features [: num_responses, :]

In [24]:
train_features.shape

(107940, 8192)

In [ ]:
clf.fit(train_features, upsampled)

In [ ]:
clf.score

In [44]:
print(train_features.mean())

0.004793217772172209

In [17]:
v1lh.shape

(490, 7200)

In [40]:
def regression_model(output_size, summary=False, backend='tf'):
    """ Return the Keras model of the multi-output ridge regression network
    """
    model = Sequential()
    model.add(Dense(8192, input_dim=8192, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(.5))
    model.add(Dense(output_size, activation='softmax'))

    if summary:
        print(model.summary())

    return model

In [41]:
def train_regressor(regressor,features,responses, opt):

    regressor.compile(loss='mean_squared_error', optimizer=opt)
    regressor.fit(features, responses)

In [42]:
regressor = regression_model(output_size = upsampled.shape[0])

In [47]:
train_regressor(regressor, train_features, upsampled)

Epoch 1/1
107940/107940 [==============================] - 2186s 20ms/step - loss: 45.9710


In [44]:
regressor

In [51]:
test_features = np.load(extracted_features_path_test)

In [52]:
print(f'test features loaded with shape {test_features.shape}')

test features loaded with shape (8085, 8192)


In [118]:
test_features = test_features [:8025, :]

In [119]:
test_features.shape

(8025, 8192)

In [45]:
regressor2 = regression_model(output_size = v1lh.shape[0])

In [46]:
train_regressor(regressor2, train_features, upsampled, opt = 'adam')

Epoch 1/1
107940/107940 [==============================] - 5497s 51ms/step - loss: 1.1517


In [101]:
train_regressor(regressor2, train_features, upsampled, opt = 'adam')

Epoch 1/1
107940/107940 [==============================] - 17100s 158ms/step - loss: 0.9924


In [111]:
v1lh_test = load.load_validation_response("1", "v1lh")

In [112]:
print(f'v1lh test loaded with shape {v1lh_test.shape}')

v1lh test loaded with shape (494, 540)


In [113]:
chop_resp_test = v1lh_test[:, fmri_lag : ]

In [114]:
chop_resp_test.shape

(494, 536)

In [115]:
chop_resp_test = PreProcess.dropna(chop_resp_test)

In [116]:
chop_resp_test = np.transpose(chop_resp_test)

In [117]:
upsampled_test = PreProcess.upsample(chop_resp_test, 8025)
print(f'upsampled shape: {upsampled_test.shape}')

upsampled shape: (8025, 490)


In [125]:
upsampled_test = np.transpose(upsampled_test)

In [126]:
upsampled_test.shape

(490, 8025)

In [64]:
y_pred = regressor2.predict(test_features)

In [92]:
y_pred.shape

(490, 8085)

In [84]:
y_pred

array([[2.8291752e-04, 2.8291804e-04, 2.8291799e-04, ..., 2.8292279e-04,
        2.8292291e-04, 2.8292302e-04],
       [9.3577699e-05, 9.3577917e-05, 9.3577895e-05, ..., 9.3579525e-05,
        9.3579569e-05, 9.3579562e-05],
       [4.6661207e-05, 4.6661335e-05, 4.6661349e-05, ..., 4.6662673e-05,
        4.6662717e-05, 4.6662717e-05],
       ...,
       [3.9779778e-05, 3.9779905e-05, 3.9779938e-05, ..., 3.9781426e-05,
        3.9781440e-05, 3.9781444e-05],
       [7.5210952e-03, 7.5211511e-03, 7.5211716e-03, ..., 7.5225150e-03,
        7.5225071e-03, 7.5224899e-03],
       [4.8970203e-05, 4.8970334e-05, 4.8970353e-05, ..., 4.8971859e-05,
        4.8971873e-05, 4.8971902e-05]], dtype=float32)

In [73]:
y_pred = np.transpose(y_pred)
upsampled_test =  np.transpose(upsampled_test)

In [98]:
from sklearn.metrics import r2_score
rsquare = np.empty(490)
for i in range(y_pred.shape[0]):
    print(i)
    rsquare[i] = r2_score(upsampled_test[i], y_pred[i])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [100]:
np.average(rsquare)

-0.0012319274449261628

In [121]:
y_pred = regressor2.predict(test_features)

In [122]:
y_pred = np.transpose(y_pred)

In [123]:
y_pred.shape

(490, 8025)

In [127]:
from sklearn.metrics import r2_score
rsquare = np.empty(490)
for i in range(y_pred.shape[0]):
    print(i)
    rsquare[i] = r2_score(upsampled_test[i], y_pred[i])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [129]:
np.average(rsquare)

-0.0010129329561501983

In [142]:
v1rh = load.load_train_response("1", "v1rh")
print(f'v1rh loaded with shape {v1rh.shape}')

v1rh loaded with shape (514, 7200)


In [143]:
v1rh = PreProcess.dropna(v1rh)
print(v1rh.shape)
v1rh = v1rh[:, fmri_lag : ]
print(v1rh.shape)
v1rh_transpose = np.transpose(v1rh)
print(f'chopped response tranposed with shape {v1rh_transpose.shape}')

(504, 7200)
(504, 7196)
chopped response tranposed with shape (7196, 504)


In [145]:
upsampled = PreProcess.upsample(v1rh, 107940)
print(f'upsampled with shape: {upsampled.shape}')

upsampled with shape: (107940, 504)


In [149]:
v1rhregressor = regression_model(output_size = v1rh.shape[0])

In [137]:
train_features.shape

(107940, 8192)

In [150]:
train_regressor(v1rhregressor, train_features, upsampled, opt = 'adam')

Epoch 1/1
107940/107940 [==============================] - 3391s 31ms/step - loss: 1.1436


In [169]:
train_regressor(v1rhregressor, train_features, upsampled, opt = 'adam')

Epoch 1/1
107940/107940 [==============================] - 18661s 173ms/step - loss: 0.9870


In [170]:
y_pred = v1rhregressor.predict(test_features)
v1rh_test = load.load_validation_response("1", "v1rh")
v1rh_test = PreProcess.dropna(v1rh_test)
print(v1rh_test.shape)
v1rh_test = v1rh_test[:, fmri_lag : ]
print(v1rh_test.shape)
v1rh_test_transpose = np.transpose(v1rh_test)
print(f'chopped response tranposed with shape {v1rh_test_transpose.shape}')

(504, 540)
(504, 536)
chopped response tranposed with shape (536, 504)


In [171]:
upsampled_test = PreProcess.upsample(v1rh_test_transpose, 8025)
print(f'upsampled with shape: {upsampled_test.shape}')

upsampled with shape: (8025, 504)


rsquare = np.empty(y_pred.shape[1])
for i in range(y_pred.shape[1]):
    rsquare[i] = r2_score(upsampled_test[i], y_pred[i])

In [174]:
np.average(rsquare)

-0.2029547839300463

In [173]:
rsquare.shape

(504,)